## RL4LLM roadmap

- 从 trl 开始学起，框架较为基础和简单；
    - 深入地学习 GRPO，基于 1.5B 复现 R1，复现 aha moments；
        - https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb
        - 大致也基本能搞清楚 RLHF 阶段的 PPO 算法原理，二者在公式上主要只有 adv（advantage）的估计方法不同；
- 后续可以陆陆续续迁移到更现代更多工程性能优化的 RL4LLM 的框架上
    - 比如 veRL 和 OpenRLHF
    - 假如都是零基础，优先 veRL 吧，除非继承而来的项目是 OpenRLHF；    
    - veRL：2409.19256，3.8k stars；
        - https://github.com/Jiayi-Pan/TinyZero
        - https://github.com/agentica-project/deepscaler
        - https://github.com/Unakar/Logic-RL
    - OpenRLHF：2405.11143，5k stars；

### TRL ppo trainer

- https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_trainer.py
- make experiences
    - forward
        - queries: `[4, 56]`
        - reponses: `[4, 53]`（$\pi_{\theta_{old}}$）
        - logprobs: `[4, 53]` （$\pi_{\theta_{old}}$）
        - ref_logprobs: `[4, 53]`（$\pi_{ref}$）
        - values: `[4, 53]`
        - scores: `[4]` (last token's, the whole query + reponse)
    - 计算 rewards (token 级别)
        - $r_t = r_{T} - \beta (\log\pi_\theta-\log\pi_{ref})$
            - 内循环；
            - KL 项是 k1 近似；
    - 计算 advantage & return
        - GAE：
            - $\delta_t=r_t+\gamma V(s_{t+1})-V(s_t)$
            - $A_t=\sum_{k=0}^T(\gamma\lambda)^k\delta_{t+k}$
        - return: advantage + value
- ppo update ($\pi_\theta$)

## adv(advantage) estimator

- GAE
- GRPO
- RLOO
- REINFORCE++
- ReMax

> verl/trainer/ppo/ray_trainer.py/compute_advantage

> verl/trainer/ppo/core_algos.py

https://verl.readthedocs.io/en/latest/examples/config.html
- compute_gae_advantage_return
    - `token_level_rewards`, `values`
    - $A_t^{GAE}=\sum_{\ell}^{T-t}(\gamma\lambda)^{\ell}\delta_{t+\ell}, \quad \delta_t=r_t+\gamma V(s_{t+1})-V(s_t)$
    - return: $ret_t=V(s_t)+A_t^{GAE}$
- compute_grpo_outcome_advantage
    - `token_level_rewards`
    - $A_i=\frac{r_i-\mu}{\sigma+\epsilon}$
- compute_rloo_outcome_advantage
    - `token_level_rewards`
    - $A_i=R_i-\frac1{n-1}\sum_{k\neq i}R_k$
- compute_reinforce_plus_plus_outcome_advantage
    - `token_level_rewards`
    - $A_t=\frac{G_t-\mu}{\sigma}, \quad G_t=\sum_{k=t}^T\gamma^{k-t}r_k$
        - return: accumulate discounted reward
- compute_remax_outcome_advantage（Reward-Maximization with Baseline）
    - `token_level_rewards`, `reward_baselines`
    - $A_t=G_t-b, \quad G_t=\sum_{k=t}^Tr_k$
        - no discounted return